<a href="https://colab.research.google.com/github/manjotmb20/EC3/blob/master/ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Defining the dataset and some preprocessing for model building**

In [144]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from numpy import linalg as LA

dataset=pd.read_csv('train.csv')

dataset2=pd.read_csv('test.csv')
dataset=dataset.fillna(dataset.mean())
dataset2=dataset2.fillna(dataset2.mean())
embarked=pd.get_dummies(dataset.Embarked,prefix='Embarked')
embarked2=pd.get_dummies(dataset2.Embarked,prefix='Embarked')
dataset=pd.concat([dataset,embarked],axis=1)
dataset2=pd.concat([dataset2,embarked2],axis=1)
dataset.drop(['Cabin', 'Ticket','Embarked','Name','PassengerId'],axis=1, inplace=True)
dataset2.drop(['Cabin', 'Ticket','Embarked','Name','PassengerId'],axis=1, inplace=True)
lc=LabelEncoder()
dataset[['Sex']]=lc.fit_transform(dataset[['Sex']])
dataset2[['Sex']]=lc.transform(dataset2[['Sex']])
sc=StandardScaler()
dataset[['Fare']]=sc.fit_transform(dataset[['Fare']])
dataset2[['Fare']]=sc.transform(dataset2[['Fare']])
sc1=StandardScaler()
dataset[['Age']]=sc1.fit_transform(dataset[['Age']])
dataset2[['Age']]=sc1.transform(dataset2[['Age']])
df_test=dataset['Survived']
df_train=dataset.drop('Survived',axis=1)
test=dataset2

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [145]:
dataset2.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,3,1,0.369449,0,0,-0.490783,0,1,0
1,3,0,1.331378,1,0,-0.507479,0,0,1
2,2,1,2.485693,0,0,-0.453367,0,1,0
3,3,1,-0.207709,0,0,-0.474005,0,0,1
4,3,0,-0.592481,1,1,-0.401017,0,0,1


**We create only 2 models , 1 classifier model with Logistic regression and 1 clustering based model with Kmeans**

In [146]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score,accuracy_score
x_train,x_val,y_train,y_val=train_test_split(df_train,df_test,test_size=0.2,random_state=9)
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2,init='k-means++')
kmeans.fit(x_train,y_train)
k_pred=kmeans.predict(x_val)
accuracy_score(y_val,k_pred)
df_kmeans=pd.DataFrame()
df_kmeans['Survived_k']=k_pred
survivedk=pd.get_dummies(df_kmeans.Survived_k,prefix='Survivedk')
classifier=LogisticRegression(random_state=0)
classifier.fit(x_train,y_train)
ypred=classifier.predict(x_val)
y_prob=classifier.predict_proba(x_val)
df=pd.DataFrame()
df['pred']=ypred
classifier1=DecisionTreeClassifier()
classifier1.fit(x_train,y_train)
ypred1=classifier1.predict(x_val)
df['pred2']=ypred1
y_prob2=classifier1.predict_proba(x_val)
print(roc_auc_score(y_val,ypred1))
print(accuracy_score(y_val,ypred1))

survived=pd.get_dummies(df.pred,prefix='Survived')
survived_classifier2=pd.get_dummies(df.pred2,prefix='Survsecond')
df=pd.concat([df,survived],axis=1)
df=pd.concat([df,survived_classifier2],axis=1)
df.drop(['pred','pred2','Survsecond_0','Survsecond_1'],axis=1,inplace=True)
df['Survivedk_0']=survivedk['Survivedk_0']
df['Survivedk_1']=survivedk['Survivedk_1']

0.7481012658227849
0.7541899441340782


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


**Creating Membership matrix(mem_mat) and cooccurence matrix(cor_mat)**

In [147]:
mem_mat=df
cor_mat=mem_mat.dot(mem_mat.T)
print("Memberhip matrix")
print(mem_mat)
print("Cooccurence matrix")
print(cor_mat)

Memberhip matrix
     Survived_0  Survived_1  Survivedk_0  Survivedk_1
0             1           0            1            0
1             1           0            1            0
2             1           0            1            0
3             1           0            0            1
4             0           1            1            0
..          ...         ...          ...          ...
174           0           1            1            0
175           0           1            1            0
176           1           0            1            0
177           0           1            0            1
178           0           1            1            0

[179 rows x 4 columns]
Cooccurence matrix
     0    1    2    3    4    5    6    ...  172  173  174  175  176  177  178
0      2    2    2    1    1    2    1  ...    1    0    1    1    2    0    1
1      2    2    2    1    1    2    1  ...    1    0    1    1    2    0    1
2      2    2    2    1    1    2    1  ...    1    0  

In [148]:
objclass_mat1=pd.DataFrame(y_prob,columns=['Survived','NotSurvived'])
objclass_mat2=pd.DataFrame(y_prob2,columns=['Survived','NotSurvived'])
df_t=pd.DataFrame(objclass_mat1)
Fo=np.array(df_t)
gpclass_mat=pd.DataFrame(columns=['Survived','NotSurvived'],index=['class1surv','class1notsurv','class2surv','class2notsurv'])
prob1s=np.sum(objclass_mat1.Survived)/len(objclass_mat1)
prob1ns=np.sum(objclass_mat1.NotSurvived)/len(objclass_mat1)
prob2s=np.sum(objclass_mat2.Survived)/len(objclass_mat2)
prob2ns=np.sum(objclass_mat2.NotSurvived)/len(objclass_mat2)
list1=[]
list1.append(prob1s)
list1.append(prob1ns)
list1.append(prob2s)
list1.append(prob2ns)

list2=[]
list2.append(prob1ns)
list2.append(prob1s)
list2.append(prob2ns)
list2.append(prob2s)
gpclass_mat['Survived']=list1
gpclass_mat['NotSurvived']=list2
gpclass_mat
Fg=np.array(gpclass_mat)
Fg

array([[0.62706492, 0.37293508],
       [0.37293508, 0.62706492],
       [0.57616387, 0.42383613],
       [0.42383613, 0.57616387]])

In [149]:
kmshape=mem_mat.shape
kcshape=cor_mat.shape
print("Shape of Km",kmshape)
print("Shape of Kc",kcshape)

Shape of Km (179, 4)
Shape of Kc (179, 179)


In [0]:
Yg=np.zeros(shape=gpclass_mat.shape)
thres=0.5
for i in range(Fg.shape[0]):
  for j in range(Fg.shape[1]):
    if Fg[i][j]>thres:
      Yg[i][j]=1
Yo=np.zeros(shape=(179,2))
cal_y=np.array(mem_mat)
for i in range(Yo.shape[0]):
  Yo[i][0]=cal_y[i][0]+cal_y[i][2]
for i in range(Yo.shape[0]):
  Yo[i][1]=cal_y[i][1]+cal_y[i][3]


In [0]:
import sys
threshold=sys.float_info.epsilon

**The current loop as per algorithm1 actually is producing an infinite condition given that every operation is being performed on Khm and Khm is not getting updated so I use a normal normalization method to normalize my matrix.**

In [152]:
'''Km=np.zeros(shape=kmshape)
Kc=np.zeros(shape=kcshape)
Khm=np.array(mem_mat)
Khc=np.array(cor_mat)
while LA.norm((Km-Khm),'fro')>threshold:
  Km=Khm.copy()
  d=np.zeros(shape=(khmshape[0]))
  for i in range(khmshape[0]):
    d[i]=np.sum(Khm[i],axis=0)
  Km=Khm/d.reshape((d.shape[0],1))'''

"Km=np.zeros(shape=kmshape)\nKc=np.zeros(shape=kcshape)\nKhm=np.array(mem_mat)\nKhc=np.array(cor_mat)\nwhile LA.norm((Km-Khm),'fro')>threshold:\n  Km=Khm.copy()\n  d=np.zeros(shape=(khmshape[0]))\n  for i in range(khmshape[0]):\n    d[i]=np.sum(Khm[i],axis=0)\n  Km=Khm/d.reshape((d.shape[0],1))"

In [0]:
Km=np.zeros(shape=kmshape)
Kc=np.zeros(shape=kcshape)
Khm=np.array(mem_mat)
Khc=np.array(cor_mat)
Km=Khm
d=np.zeros(shape=(kmshape[0]))
for i in range(kmshape[0]):
  d[i]=np.sum(Km[i],axis=0)

Km=Km/d.reshape((d.shape[0],1))

In [154]:
Kc=np.zeros(shape=kcshape)
Khc=np.array(cor_mat)
Kc=Khc.copy()
d=np.zeros(shape=(kcshape[0]))
for i in range(kcshape[0]):
  d[i]=np.sum(Kc[i],axis=0)

Kc=Kc/d.reshape((d.shape[0],1))
Kc

array([[0.00806452, 0.00806452, 0.00806452, ..., 0.00806452, 0.        ,
        0.00403226],
       [0.00806452, 0.00806452, 0.00806452, ..., 0.00806452, 0.        ,
        0.00403226],
       [0.00806452, 0.00806452, 0.00806452, ..., 0.00806452, 0.        ,
        0.00403226],
       ...,
       [0.00806452, 0.00806452, 0.00806452, ..., 0.00806452, 0.        ,
        0.00403226],
       [0.        , 0.        , 0.        , ..., 0.        , 0.01818182,
        0.00909091],
       [0.00512821, 0.00512821, 0.00512821, ..., 0.00512821, 0.00512821,
        0.01025641]])

**Condition Check for Fo and Fg i.e |Fo|=1 for all elements of Fo and similar for Fg**

In [155]:
Fo_cond=True
Fg_cond=True
for i in range(Fo.shape[0]):
  if np.linalg.norm((Fo[i]), ord=1)!=1:
    Fo_cond=False
for i in range(Fg.shape[0]):
  if np.linalg.norm((Fg[i]), ord=1)!=1:
    Fo_cond=False    
print(Fo_cond)
print(Fg_cond)

True
True


Similarly for Yo and Yg we can see the norm to be 1 for every index

In [156]:
np.linalg.norm((Yg[1]), ord=1)

1.0

In [157]:
J1=0
for i in range(Khm.shape[0]):
  for j in range(Khm.shape[1]):
    J1=J1+Khm[i][j]*(np.linalg.norm((Fo[i]-Fg[j]), ord=2)**2)
J2=0
for i in range(Khc.shape[0]):
  for j in range(Khc.shape[1]):
    J2=J2+Khc[i][j]*(np.linalg.norm((Fo[i]-Fo[j]), ord=2)**2)    
J3=0
for i in range(Khc.shape[0]):
  J3=J3+(np.linalg.norm((Fo[i]-Yo[i]), ord=2)**2) 

J4=0
for i in range(Khm.shape[1]):
  J4=J4+(np.linalg.norm((Fg[i]-Yg[i]), ord=2)**2) 
print(J1)
print(J2)
print(J3)
print(J4)

46.015169221771814
6794.730305842706
202.238970517923
1.2748705592944716


Our goal is to minimize these values

Calculating diagonal matrix Dm and Dc

In [158]:
alpha=0.20
beta=0.35
gamma=0.45
delta=0
test_dm=np.zeros(shape=(Km.shape))
test_Km=Km.T
for i in range(test_Km.shape[0]):
  test_dm[i]=np.sum(test_Km[i],axis=0)
Dm=np.diag(np.diag(Km.T))
print("Shape of DM",Dm.shape)
test_dc=np.zeros(shape=(Kc.shape))
test_Kc=Kc.T
for i in range(test_Kc.shape[0]):
  test_dc[i]=np.sum(test_Kc[i],axis=0)
Dc=np.diag(np.diag(test_dc.T))
print("shape of Dc",Dc.shape)

Shape of DM (4, 4)
shape of Dc (179, 179)


Now we implement algorithm 2, where we take Fo(t) as Fo_t and we do not take any timestamp to 

In [0]:
Fo_t=np.zeros(shape=Fo.shape)
Fg_t=np.zeros(shape=Fg.shape)
Fot_old=np.zeros(shape=Fo.shape)
Fo_t=Fo
Fg_t=Fg
while LA.norm((Fo_t-Fot_old),'fro')>threshold:
  Fot_old=Fo_t
  Fg_t=np.matmul(np.linalg.inv(2*np.ones(shape=Dm.shape)+(alpha*Dm)),(alpha*np.matmul(Km.T,Fo))+(2*delta*Yg))
  Fo_t=np.matmul((np.linalg.inv((2*beta*Dc)-(beta*np.matmul(np.identity(Kc.shape[0]),Kc))-(beta*np.matmul(np.ones(shape=Kc.shape),Kc))+(2*gamma*np.identity(Kc.shape[0])))),((alpha*np.matmul(Km,Fg_t))+(2*gamma*Yo)))

In [169]:
Fo_t

array([[  6.51951848,  -1.81135417],
       [  6.51951848,  -1.81135417],
       [  6.51951848,  -1.81135417],
       [  1.73357462,  -2.63844032],
       [ -9.90374469,   4.10888366],
       [  6.51951848,  -1.81135417],
       [  1.73357462,  -2.63844032],
       [  6.51951848,  -1.81135417],
       [  6.51951848,  -1.81135417],
       [  6.51951848,  -1.81135417],
       [  6.51951848,  -1.81135417],
       [-18.12221964,   4.26892881],
       [  6.51951848,  -1.81135417],
       [  6.51951848,  -1.81135417],
       [-18.12221964,   4.26892881],
       [  1.73357462,  -2.63844032],
       [  6.51951848,  -1.81135417],
       [ -9.90374469,   4.10888366],
       [ -9.90374469,   4.10888366],
       [  6.51951848,  -1.81135417],
       [  6.51951848,  -1.81135417],
       [  6.51951848,  -1.81135417],
       [ -9.90374469,   4.10888366],
       [  6.51951848,  -1.81135417],
       [  6.51951848,  -1.81135417],
       [  6.51951848,  -1.81135417],
       [  6.51951848,  -1.81135417],
 

Result of the given model

In [171]:
result=np.zeros(shape=Fo_t.shape[0])
for i in range(Fo_t.shape[0]):
  if Fo_t[i][0]<Fo_t[i][1]:
    result[i]=1
print("AUC value",roc_auc_score(y_val,result))
print("AUC for model1",roc_auc_score(y_val,ypred1))

AUC value 0.7627848101265823
AUC for model1 0.7481012658227849
